In [3]:
import sys
import os
from pathlib import Path

import findspark
findspark.init()
findspark.find()

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


os.environ['PYSPARK_PYTHON'] = 'venv/bin/python'


spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Spark Streaming") \
    .config("spark.driver.extraJavaOptions", "-Dcom.sun.net.ssl.checkRevocation=false") \
    .config("spark.executor.extraJavaOptions", "-Dcom.sun.net.ssl.checkRevocation=false") \
    .config("spark.executor.extraJavaOptions", "-Dhttps.protocols=TLSv1.1,TLSv1.2") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.0.1") \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true") \
    .config("spark.yarn.dist.archives", f"{Path('/home', os.environ.get('USER'))}/environment.tar.gz#venv") \
    .config("spark.executor.instances", "1") \
    .config("spark.executor.memory", "512mb") \
    .getOrCreate()

def kill_all():
    streams = SparkSession.builder.getOrCreate().streams.active
    if streams:
        for stream in streams:
            desc = stream.lastProgress["sources"][0]["description"]
            stream.stop()
            print("Stopped {s}".format(s=desc))

In [9]:
cass_df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="t_test", keyspace="ks_test") \
    .load()

In [10]:
cass_df.show()

+----------+---+--------------------+--------------------+
|patient_id| id|                date|             details|
+----------+---+--------------------+--------------------+
|         1|  1|feca2000-dc4b-11e...|first exam patient 1|
|         1|  2|fecbf4c0-dc4b-11e...|second exam patie...|
|         3|  1|fecd2d40-dc4b-11e...|first exam patient 3|
|         2|  1|fecc9100-dc4b-11e...|first exam patient 2|
+----------+---+--------------------+--------------------+

